In [27]:
import pandas as pd
import requests
import io
import  numpy as np
import matplotlib.pyplot as plt
import pickle5
import  json
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 80

target_list_v6 = pd.read_csv('speculoos_target_list_v6.txt',sep=' ')
target_list_v6.columns

Index(['Sp_ID', '2MASS', 'Gaia_ID', 'wise', 'Program', 'nb_hours_surved',
       'nb_hours_threshold', 'RA', 'DEC', 'e_gra', 'e_gdec', 'telescope',
       'Filter_spc', 'texp_spc', 'Filter_trap', 'texp_trap', 'dist',
       'dist_err', 'SpT', 'e_SpT', 'Ms', 'e_Ms', 'Rs', 'e_Rs', 'T_eff',
       'teff_err', 'G', 'G_err', 'I', 'I_err', 'J', 'J_err', 'H', 'H_err', 'K',
       'K_err', 'SNR_TESS_temp', 'SNR_Spec_temp', 'SNR_TESS_HZ', 'SNR_Spec_HZ',
       'SNR_JWST_HZ_tr', 'SNR_JWST_HZ_occ', 'SNR_JWST_temp_occ', 'Note',
       'Known_Binary', 'mag_bp', 'mag_bp_err', 'mag_rp', 'mag_rp_err', 'Habs',
       'e_Habs', 'Jabs', 'e_Jabs', 'Kabs', 'e_Kabs', 'Gabs', 'e_Gabs', 'BCj',
       'BCj_err', 'L_bol', 'L_bol_err', 'cf', 'num_stars', 'min_dist_stars',
       'avg_dist_stars', 'rel_flux_stars', 'plx', 'plx_err', 'pmra', 'pmdec',
       'pmra_err', 'pmdec_err'],
      dtype='object')

In [28]:
a = target_list_v6.where((target_list_v6['Sp_ID']=='Sp0025+5422'))
a  = a.dropna()
a

,Sp_ID,2MASS,Gaia_ID,wise,Program,nb_hours_surved,nb_hours_threshold,RA,DEC,e_gra,e_gdec,telescope,Filter_spc,texp_spc,Filter_trap,texp_trap,dist,dist_err,SpT,e_SpT,Ms,e_Ms,Rs,e_Rs,T_eff,teff_err,G,G_err,I,I_err,J,J_err,H,H_err,K,K_err,SNR_TESS_temp,SNR_Spec_temp,SNR_TESS_HZ,SNR_Spec_HZ,SNR_JWST_HZ_tr,SNR_JWST_HZ_occ,SNR_JWST_temp_occ,Note,Known_Binary,mag_bp,mag_bp_err,mag_rp,mag_rp_err,Habs,e_Habs,Jabs,e_Jabs,Kabs,e_Kabs,Gabs,e_Gabs,BCj,BCj_err,L_bol,L_bol_err,cf,num_stars,min_dist_stars,avg_dist_stars,rel_flux_stars,plx,plx_err,pmra,pmdec,pmra_err,pmdec_err
40,Sp0025+5422,00251602+5422547,4.196244e+17,J002515.62+542250.9,1.0,66.71,200.0,6.315766,54.381918,0.065499,0.082527,['Saint-Ex'],I+z,40.0,I+z,0.0,16.8,0.0,6.9,1.1,0.094,0.001,0.123,0.01,2695.0,105.0,15.741,0.000549,14.938,0.278098,11.78,0.024,11.21,0.03,10.82,0.022,9.16,14.64,6.48,11.34,4.71,1.54,4.76,add_note,0.0,18.839205,0.021953,14.21898,0.003892,10.080883,0.030301,10.650883,0.024375,9.690883,0.022409,14.612282,0.004296,1.950062,0.018868,0.000717,0.00002,0.22,40.0,15.28,0.0,0.0,0.0,0.0,-342.745,-376.195,0.139,0.134


In [29]:
idx_targets_observed = np.where(((target_list_v6['nb_hours_surved']>0) & (target_list_v6['Program']!=5)))[0]
len(idx_targets_observed )

271

### coverage

In [30]:
def index_list1_list2(list1, list2): #list 2 longer than list 1
    idx_list1_in_list2 = []
    idx_list2_in_list1 = []
    for i in range(len(list2)):
        for j in range(len(list1)):
            if list2[i] == list1[j]:
                idx_list1_in_list2.append(i)
                idx_list2_in_list1.append(j)
    return idx_list1_in_list2, idx_list2_in_list1

def getSPClcV2(target, ap = '', pwvCorr = 0, user="educrot", password="58JMSGgdmzTB"):
    urlGet = "http://www.mrao.cam.ac.uk/SPECULOOS/portal/get_tls_prep_v2.php?date=*&id=" + target + "&filter=&telescope=&ap=" + str(ap) + "&pwvCorr=" + str(pwvCorr)
    rGET = requests.get(urlGet, auth=(user, password))
    names = ['TMID-2450000', 'BJDMID-2450000', 'DIFF_FLUX', 'ERROR', 'DIFF_FLUX_PWV', 'RA_MOVE', 'DEC_MOVE', 'FWHM', 'PSF_a_5', 'PSF_b_5', 'SKYLEVEL', 'AIRMASS', 'EXPOSURE']
    lc = pd.read_csv(io.StringIO(rGET.text), header=None, names=names, delimiter='\s+')
    return lc

def getSPCdata(target, date, ap=6, user="educrot", password="58JMSGgdmzTB"):
    urlGet = "http://www.mrao.cam.ac.uk/SPECULOOS/portal/get_file.php?telescope=&date=" + date + "&id=" + target + "&filter=&file=MCMC_text_" + str(ap)
    rGET = requests.get(urlGet, auth=(user, password))
    rFile = json.loads(rGET.text)
    print(rFile)
#     print(rFile)
    # retrieve available file
    try:
        urlMCMC = 'http://www.mrao.cam.ac.uk/SPECULOOS/portal/' + rFile[0]
        rMCMC = requests.get(urlMCMC, auth=(user, password))
        # read files
        MCMC = pd.read_csv(io.StringIO(rMCMC.text), sep=' ')
        MCMC['JD'] = MCMC['TMID-2450000'] + 2450000
        targetdf = pd.DataFrame({'JD':MCMC['JD'].values,'DIFF_FLUX': MCMC['DIFF_FLUX'].values, 'ERROR': MCMC['ERROR'].values, 'AIRMASS': MCMC['AIRMASS'].values})
        return targetdf
    except TypeError:
        targetdf = pd.DataFrame({'JD':[],'DIFF_FLUX': [], 'ERROR': [], 'AIRMASS': []})
        return targetdf.reset_index(drop=True)


night = getSPCdata(target='TOI-736',date='*',ap='5')
night

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def coverage(t, p):
    if p==0:
        return 1
    else:
        ph = ((t + 0.5*p)%p - (0.5*p))
        sph_in = np.sort(ph)
        sph_out = sph_in[::-1]

        sph_in_diff = np.abs(np.diff(sph_in))
        sph_out_diff = np.abs(np.diff(sph_out))

        df = np.min(np.diff(t))

        spaces_in = np.sort(sph_in[np.hstack([*np.argwhere(sph_in_diff > 4*df).T, len(sph_in)-1])])
        spaces_out = np.sort(sph_out[np.hstack([*np.argwhere(sph_out_diff > 4*df).T, len(sph_in)-1])])

        return np.sum(spaces_in - spaces_out)/p

    
def phase_coverage_given_target(name_observatory,target,path_target_list,pmin,pmax):

    if path_target_list is None:
        path_target_list = 'speculoos_target_list_v6.txt'

    target_list = pd.read_csv(path_target_list,sep=' ')
    idx_target_list = list(target_list['Sp_ID']).index(target)


    #data = getSPCdata(target,date='*',ap=5)
    data = getSPClcV2(target, ap = '', pwvCorr = 0)

    t = data['BJDMID-2450000'] # 'BJDMID-2450000' # 'JD'
    t = t.fillna(0)

    P_min = pmin
    P_max = pmax
    periods = np.arange(P_min, P_max, 0.01)

    try:
        covs = [coverage(t, period) for period in periods]
        mean_cov = np.mean(covs)*100
        return covs,mean_cov,target_list['nb_hours_surved'][idx_target_list]
        #fig, ax = plt.subplots(1, figsize=(9, 7))
        #plt.plot(periods, covs, c="silver",label='Effective cov = ' +  str(round(mean_cov,1)) + ' %')
        #plt.plot(periods, covs, ".", c="k",label=r'$SNR_{JWST}$ = ' +  str(round(target_list['SNR_JWST_HZ_tr'][idx_target_list],3)))
        #plt.ylabel('Phase coverage in %')
        #plt.xlabel('Period in days')
        #plt.legend(fontsize=16)
        #plt.grid(color='gainsboro', linestyle='-', linewidth=1, alpha=0.3)
        #plt.title(r'Phase coverage for ' + target + ' with periods $\in$ ' + str(pmin) + ' - ' + str(pmax) )
        #plt.show()
    except ValueError:
        return 0,0,target_list['nb_hours_surved'][idx_target_list]
        print('ERROR: No v2 data for this target ! ')


In [ ]:
night = getSPCdata(target='trappist-1',date='20180620',ap='5')

In [ ]:
idx_targets_observed = np.where((target_list_v6['nb_hours_surved']>10))[0]
#idx_targets_observed = idx_targets_observed[44:55]
mean_cov_observed_targets = []
nb_hours_target_observed = []

for i in idx_targets_observed:
    covs,mean_cov,nb_hours = phase_coverage_given_target(name_observatory='SSO',target=target_list_v6['Sp_ID'][i],
                                    path_target_list=None,pmin=0,pmax=6) 
    mean_cov_observed_targets.append(mean_cov)
    nb_hours_target_observed.append(target_list_v6['nb_hours_surved'][i])
    print(target_list_v6['Sp_ID'][i],mean_cov,nb_hours)





In [ ]:
idx_targets_observed = np.where(((target_list_v6['nb_hours_surved']>50) & (target_list_v6['Program']!=5)))[0]

lists = np.array_split(idx_targets_observed, 5)
list_1 = lists[0]
list_2 = lists[1]
list_3 = lists[2]
list_4 = lists[3]
list_5 = lists[4]

df_list1 = pd.DataFrame({'Sp_ID':target_list_v6['Sp_ID'][list_1],'Gaia_ID':target_list_v6['Gaia_ID'][list_1],'nb_hours_surved':target_list_v6['nb_hours_surved'][list_1],
                       'Program':target_list_v6['Program'][list_1],'RA':target_list_v6['RA'][list_1],'DEC':target_list_v6['DEC'][list_1],
                        'Telescope_used':target_list_v6['telescope'][list_1],'Notes':target_list_v6['Note'][list_1],
                        'observation_dates':dates_obs_list1_merged 
                        })
df_list1.to_csv('List1.csv',sep=',',index=None)

df_list2 = pd.DataFrame({'Sp_ID':target_list_v6['Sp_ID'][list_2],'Gaia_ID':target_list_v6['Gaia_ID'][list_2],'nb_hours_surved':target_list_v6['nb_hours_surved'][list_2],
                       'Program':target_list_v6['Program'][list_2],'RA':target_list_v6['RA'][list_2],'DEC':target_list_v6['DEC'][list_2],
                        'Telescope_used':target_list_v6['telescope'][list_2],'Notes':target_list_v6['Note'][list_2],
                         'observation_dates':dates_obs_list2_merged
                        })
df_list2.to_csv('List2.csv',sep=',',index=None)

df_list3 = pd.DataFrame({'Sp_ID':target_list_v6['Sp_ID'][list_3],'Gaia_ID':target_list_v6['Gaia_ID'][list_3],'nb_hours_surved':target_list_v6['nb_hours_surved'][list_3],
                       'Program':target_list_v6['Program'][list_3],'RA':target_list_v6['RA'][list_3],'DEC':target_list_v6['DEC'][list_3],
                        'Telescope_used':target_list_v6['telescope'][list_3],'Notes':target_list_v6['Note'][list_3],
                         'observation_dates':dates_obs_list3_merged
                        })
df_list3.to_csv('List3.csv',sep=',',index=None)

df_list4 = pd.DataFrame({'Sp_ID':target_list_v6['Sp_ID'][list_4],'Gaia_ID':target_list_v6['Gaia_ID'][list_4],'nb_hours_surved':target_list_v6['nb_hours_surved'][list_4],
                       'Program':target_list_v6['Program'][list_4],'RA':target_list_v6['RA'][list_4],'DEC':target_list_v6['DEC'][list_4],
                        'Telescope_used':target_list_v6['telescope'][list_4],'Notes':target_list_v6['Note'][list_4],
                         'observation_dates':dates_obs_list4_merged
                        })
df_list4.to_csv('List4.csv',sep=',',index=None)

df_list5 = pd.DataFrame({'Sp_ID':target_list_v6['Sp_ID'][list_5],'Gaia_ID':target_list_v6['Gaia_ID'][list_5],'nb_hours_surved':target_list_v6['nb_hours_surved'][list_5],
                       'Program':target_list_v6['Program'][list_5],'RA':target_list_v6['RA'][list_5],'DEC':target_list_v6['DEC'][list_5],
                        'Telescope_used':target_list_v6['telescope'][list_5],'Notes':target_list_v6['Note'][list_5],
                         'observation_dates':dates_obs_list5_merged
                        })
df_list5.to_csv('List5.csv',sep=',',index=None)

df_list4

In [ ]:
frames = [df_list1,df_list2,df_list3,df_list4,df_list5]

all_targets = pd.concat(frames)

all_targets

### Get dates of obs for each target observed

In [ ]:
import glob
import os

def give_dates_of_obs(target):
    dates_of_obs = []
    telescopes = ['Artemis','Saint-Ex','Io','Europa','Ganymede','Callisto','TS_La_Silla','TN_Oukaimeden']
    for tel in telescopes:
        days = glob.glob('./DATABASE/' + tel + '/*.zip')
        dates = [days[i].replace('./DATABASE/' + tel + '/','').replace('.zip','') for i in range(len(days))]
        for d in dates:
            plans = glob.glob('./DATABASE/' + tel + '/Plans_by_date/' + d  + '/*.txt')
            for p in plans:
                if p.endswith('Obj_' + target + '.txt'):
                    dates_of_obs.append(tel + '_' + d)

    return dates_of_obs

#dates_obs_list1 = []
#for t in target_list_v6['Sp_ID'][list_1]:
#    dates_obs_list1.append(give_dates_of_obs(target=t))
    
#dates_obs_list2 = []
#for t in target_list_v6['Sp_ID'][list_2]:
#    dates_obs_list2.append(give_dates_of_obs(target=t))

dates_obs_list3 = []
for t in target_list_v6['Sp_ID'][list_3]:
    dates_obs_list3.append(give_dates_of_obs(target=t))
    
dates_obs_list4 = []
for t in target_list_v6['Sp_ID'][list_4]:
    dates_obs_list4.append(give_dates_of_obs(target=t))
    
dates_obs_list5 = []
for t in target_list_v6['Sp_ID'][list_5]:
    dates_obs_list5.append(give_dates_of_obs(target=t))

In [ ]:
def getSPCdata_elsa(target, date, ap=6, user="educrot", password="58JMSGgdmzTB"):
    urlGet = "http://www.mrao.cam.ac.uk/SPECULOOS/portal/get_file.php?telescope=&date=" + date + "&id=" + target + "&filter=&file=MCMC_text_" + str(ap)
    rGET = requests.get(urlGet, auth=(user, password))
    rFile = json.loads(rGET.text)
    dates = []
    targetdf = pd.DataFrame({'JD':[],'DIFF_FLUX': [], 'ERROR': [], 'AIRMASS': []})
    # retrieve available file
    try:
        for i in range(len(rFile)):
            if rFile[i].find('Europa') != -1:
                tel = 'Europa'
            if rFile[i].find('Io') != -1:
                tel = 'Io'
            if rFile[i].find('Ganymede') != -1:
                tel = 'Ganymede'
            if rFile[i].find('Callisto') != -1:
                tel = 'Callisto'
            if rFile[i].find('TN_Oukaimeden') != -1:
                tel = 'TN_Oukaimeden'
            if rFile[i].find('TS_La_Silla') != -1:
                tel = 'TS_La_Silla'
            d = rFile[i].replace('..','').replace('/output/','').replace('/lightcurves/I+z/MCMC_text_6','').replace(str(target),'').replace('/','').replace('Io','').replace('Europa','').replace('Ganymede','').replace('Callisto','').replace('Artemis','')
            dates.append(tel + '_' + d)
            try:
                urlMCMC = 'http://www.mrao.cam.ac.uk/SPECULOOS/portal/' + rFile[i]
                rMCMC = requests.get(urlMCMC, auth=(user, password))
                # read files
                MCMC = pd.read_csv(io.StringIO(rMCMC.text), sep=' ')
                MCMC['JD'] = MCMC['TMID-2450000'] + 2450000
                frames = [targetdf,MCMC]
                targetdf = pd.concat(frames)
                #targetdf.append(pd.DataFrame({'JD':MCMC['JD'].values,'DIFF_FLUX': MCMC['DIFF_FLUX'].values,
                #                               'ERROR': MCMC['ERROR'].values, 'AIRMASS': MCMC['AIRMASS'].values}))
                #return targetdf
            except TypeError:
                targetdf.append(pd.DataFrame({'JD':[],'DIFF_FLUX': [], 'ERROR': [], 'AIRMASS': []}))
                #return targetdf.reset_index(drop=True)
    except TypeError:
        print('WARNING: No obs')
    return targetdf, dates
    


In [ ]:
from progressbar import ProgressBar
pbar = ProgressBar()
dates_list_3 = []
for i in pbar(range(len(list_3))):
    a,d =getSPCdata_elsa(target=target_list_v6['Sp_ID'][list_3[i]], date="*", ap=6)
    dates_list_3.append(d)
    



In [ ]:
pbar = ProgressBar()

dates_list_4 = []
for i in pbar(range(len(list_4))):
    a,d =getSPCdata_elsa(target=target_list_v6['Sp_ID'][list_4[i]], date="*", ap=6)
    dates_list_4.append(d)

pbar = ProgressBar()
dates_list_5 = []
for i in pbar(range(len(list_5))):
    a,d =getSPCdata_elsa(target=target_list_v6['Sp_ID'][list_5[i]], date="*", ap=6)
    dates_list_5.append(d)

In [ ]:
result = []
for i in range(len(list_3)):
    in_second_but_not_in_first = set(dates_obs_list3[i]) - set(dates_list_3[i])
    result.append(dates_list_3[i] + list(in_second_but_not_in_first))
    result
    
dates_obs_list3_merged = result

result = []
for i in range(len(list_4)):
    in_second_but_not_in_first = set(dates_obs_list4[i]) - set(dates_list_4[i])
    result.append(dates_list_4[i] + list(in_second_but_not_in_first))
    result
    
dates_obs_list4_merged = result

result = []
for i in range(len(list_5)):
    in_second_but_not_in_first = set(dates_obs_list5[i]) - set(dates_list_5[i])
    result.append(dates_list_5[i] + list(in_second_but_not_in_first))
    result
    
dates_obs_list5_merged = result

In [ ]:
dates_obs_list4_merged 

### Random

In [ ]:
import random

#random.shuffle()

index = list(all_targets['Sp_ID'].index)
random.shuffle(index)

index

In [ ]:
24/4

In [ ]:
import more_itertools as mit

windows = list(mit.windowed(index, n=40, step=25))

#for i in range(0,24):
 #   print(windows[-1][26+i],windows[0][i])
    #windows[-1][25+i] = windows[0][i]

    
#list1 = np.array(windows[0])
#list2 = np.array(windows[1])
#list3 = np.array(windows[2])
#list4 = np.array(windows[3])
#list5 = np.array(windows[4])
#list6 = np.array(windows[5])
#list7 = np.array(windows[6][:-4])

#list5,list6,len(windows),list7

list1_a = index[:24]
list2_a = index[24:48] 
list3_a = index[48:72]
list4_a = index[72:96]
list5_a = index[96:120] 
list6_a = index[120:144] 
list7_a = index[144:168] 
list8_a = index[168:186] 

list1 = list1_a + list2_a[:6] + list4_a[6:12] + list6_a[12:18] + list8_a[18:24] 
list2 = list2_a + list1_a[:6] + list3_a[6:12] + list5_a[12:18] + list7_a[18:24]
list3 = list3_a + list4_a[:6] + list6_a[6:12] + list8_a[12:18] + list2_a[18:24]
list4 = list4_a + list5_a[:6] + list7_a[6:12] + list1_a[12:18] + list3_a[18:24]
list5 = list5_a + list6_a[:6] + list8_a[6:12] + list2_a[12:18] + list4_a[18:24]
list6 = list6_a + list7_a[:6] + list1_a[6:12] + list3_a[12:18] + list5_a[18:24]
list7 = list7_a + list8_a[:6] + list2_a[6:12] + list4_a[12:18] + list6_a[18:24]
list8 = list8_a + list3_a[:6] + list5_a[6:12] + list7_a[12:18] + list1_a[18:24]


len(list7),len(list1_a),len(list8)

In [ ]:
i = 476
for j in range(len(windows)):
    ici = np.where((np.array(windows[j]) ==i))[0]
    print(ici)

In [ ]:
for i in list6:
    if i in list7 :
        print(i)


In [ ]:
list7 = np.array(windows[0][:29]+windows[-1][15:-4])

In [ ]:
#a= np.load('/Users/elsaducrot/specphot/coverage/coverage_info_per_target.npy',allow_pickle=True)

with open('/Users/elsaducrot/Documents/PhD_SPECULOOS/WG2/a_storage_0_to_202.pick','rb') as f:
    a1 = pickle.load(f)
with open('/Users/elsaducrot/Documents/PhD_SPECULOOS/WG2/a_storage_202_to_203.pick','rb') as f:
    a2 = pickle.load(f)
with open('/Users/elsaducrot/Documents/PhD_SPECULOOS/WG2/a_storage_203_to_291.pick','rb') as f:
    a3 = pickle.load(f)
with open('/Users/elsaducrot/Documents/PhD_SPECULOOS/WG2/a_storage_291_to_292.pick','rb') as f:
    a4 = pickle.load(f)
with open('/Users/elsaducrot/Documents/PhD_SPECULOOS/WG2/a_storage_291_to_end.pick','rb') as f:
    a5 = pickle.load(f)
    
a = np.concatenate((a1,a2,a3,a4,a5), axis=0)


In [ ]:
name_a = [a[i][0] for i in range(0,len(a))]

idx_a_in_v6,idx_v6_in_a = index_list1_list2(name_a,target_list_v6['Sp_ID'])

In [ ]:

t = np.concatenate(a[idx_v6_in_a[1]][1])
fig = plt.subplots(figsize=(20,3))
flux = [1]*len(t)
plt.ylim([0.99,1.01])
plt.plot(np.array(t)-2450000,flux,'.',markersize=10)


In [ ]:
len(list1),len(list2),len(list3),len(list4),len(list5),len(list6),len(list7)

In [ ]:
list6

In [ ]:


df_2_list1 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list1],'Gaia_ID':target_list_v6['Gaia_ID'][list1],'nb_hours_surved':all_targets['nb_hours_surved'][list1],
                       'Program':all_targets['Program'][list1],'RA':all_targets['RA'][list1],'DEC':all_targets['DEC'][list1],
                        'Telescope_used':all_targets['Telescope_used'][list1],'Notes':all_targets['Notes'][list1],
                        'observation_dates':all_targets['observation_dates'][list1]
                        })
df_2_list1.to_csv('List1.csv',sep=',',index=None)

df_2_list2 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list2],'Gaia_ID':target_list_v6['Gaia_ID'][list2],'nb_hours_surved':all_targets['nb_hours_surved'][list2],
                       'Program':all_targets['Program'][list2],'RA':all_targets['RA'][list2],'DEC':all_targets['DEC'][list2],
                        'Telescope_used':all_targets['Telescope_used'][list2],'Notes':all_targets['Notes'][list2],
                         'observation_dates':all_targets['observation_dates'][list2]
                        })
df_2_list2.to_csv('List2.csv',sep=',',index=None)

df_2_list3 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list3],'Gaia_ID':target_list_v6['Gaia_ID'][list3],'nb_hours_surved':all_targets['nb_hours_surved'][list3],
                       'Program':all_targets['Program'][list3],'RA':all_targets['RA'][list3],'DEC':all_targets['DEC'][list3],
                        'Telescope_used':all_targets['Telescope_used'][list3],'Notes':all_targets['Notes'][list3],
                         'observation_dates':all_targets['observation_dates'][list3]
                        })
df_2_list3.to_csv('List3.csv',sep=',',index=None)

df_2_list4 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list4],'Gaia_ID':target_list_v6['Gaia_ID'][list4],'nb_hours_surved':all_targets['nb_hours_surved'][list4],
                       'Program':all_targets['Program'][list4],'RA':all_targets['RA'][list4],'DEC':all_targets['DEC'][list4],
                        'Telescope_used':all_targets['Telescope_used'][list4],'Notes':all_targets['Notes'][list4],
                         'observation_dates':all_targets['observation_dates'][list4]
                        })
df_2_list4.to_csv('List4.csv',sep=',',index=None)

df_2_list5 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list5],'Gaia_ID':target_list_v6['Gaia_ID'][list5],'nb_hours_surved':all_targets['nb_hours_surved'][list5],
                       'Program':all_targets['Program'][list5],'RA':all_targets['RA'][list5],'DEC':all_targets['DEC'][list5],
                        'Telescope_used':all_targets['Telescope_used'][list5],'Notes':all_targets['Notes'][list5],
                         'observation_dates':all_targets['observation_dates'][list5]
                        })
df_2_list5.to_csv('List5.csv',sep=',',index=None)

df_2_list6 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list6],'Gaia_ID':target_list_v6['Gaia_ID'][list6],'nb_hours_surved':all_targets['nb_hours_surved'][list6],
                       'Program':all_targets['Program'][list6],'RA':all_targets['RA'][list6],'DEC':all_targets['DEC'][list6],
                        'Telescope_used':all_targets['Telescope_used'][list6],'Notes':all_targets['Notes'][list6],
                         'observation_dates':all_targets['observation_dates'][list6]
                        })
df_2_list6.to_csv('List6.csv',sep=',',index=None)

df_2_list7 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list7],'Gaia_ID':target_list_v6['Gaia_ID'][list7],'nb_hours_surved':all_targets['nb_hours_surved'][list7],
                       'Program':all_targets['Program'][list7],'RA':all_targets['RA'][list7],'DEC':all_targets['DEC'][list7],
                        'Telescope_used':all_targets['Telescope_used'][list7],'Notes':all_targets['Notes'][list7],
                         'observation_dates':all_targets['observation_dates'][list7]
                        })
df_2_list7.to_csv('List7.csv',sep=',',index=None)


df_2_list8 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list8],'Gaia_ID':target_list_v6['Gaia_ID'][list8],'nb_hours_surved':all_targets['nb_hours_surved'][list8],
                       'Program':all_targets['Program'][list8],'RA':all_targets['RA'][list8],'DEC':all_targets['DEC'][list8],
                        'Telescope_used':all_targets['Telescope_used'][list8],'Notes':all_targets['Notes'][list8],
                         'observation_dates':all_targets['observation_dates'][list8]
                        })
df_2_list8.to_csv('List8.csv',sep=',',index=None)


df_2_list8

In [ ]:
import random
df_2_list1 = pd.read_csv('List1.csv',sep=',')
df_2_list2 = pd.read_csv('List2.csv',sep=',')
df_2_list3 = pd.read_csv('List3.csv',sep=',')
df_2_list4 = pd.read_csv('List4.csv',sep=',')
df_2_list5 = pd.read_csv('List5.csv',sep=',')
df_2_list6 = pd.read_csv('List6.csv',sep=',')
df_2_list7 = pd.read_csv('List7.csv',sep=',')
df_2_list8 = pd.read_csv('List8.csv',sep=',')

list9_a_sp = random.sample(list(df_2_list1.index.values),5) 

a = pd.DataFrame({'Sp_ID':df_2_list1['Sp_ID'][list9_a_sp],'Gaia_ID':target_list_v6['Gaia_ID'][list9_a_sp],'nb_hours_surved':df_2_list1['nb_hours_surved'][list9_a_sp],
                       'Program':df_2_list1['Program'][list9_a_sp],'RA':df_2_list1['RA'][list9_a_sp],'DEC':df_2_list1['DEC'][list9_a_sp],
                        'Telescope_used':df_2_list1['Telescope_used'][list9_a_sp],'Notes':df_2_list1['Notes'][list9_a_sp],
                         'observation_dates':df_2_list1['observation_dates'][list9_a_sp]
                        })
list9_b_sp = random.sample(list(df_2_list2.index.values),5) 

b = pd.DataFrame({'Sp_ID':df_2_list2['Sp_ID'][list9_b_sp],'Gaia_ID':target_list_v6['Gaia_ID'][list9_b_sp],'nb_hours_surved':df_2_list2['nb_hours_surved'][list9_b_sp],
                       'Program':df_2_list2['Program'][list9_b_sp],'RA':df_2_list2['RA'][list9_b_sp],'DEC':df_2_list2['DEC'][list9_b_sp],
                        'Telescope_used':df_2_list2['Telescope_used'][list9_b_sp],'Notes':df_2_list2['Notes'][list9_b_sp],
                         'observation_dates':df_2_list2['observation_dates'][list9_b_sp]
                        })

list9_c_sp = random.sample(list(df_2_list3.index.values),5) 

c = pd.DataFrame({'Sp_ID':df_2_list3['Sp_ID'][list9_c_sp],'Gaia_ID':target_list_v6['Gaia_ID'][list9_c_sp],'nb_hours_surved':df_2_list3['nb_hours_surved'][list9_c_sp],
                       'Program':df_2_list3['Program'][list9_c_sp],'RA':df_2_list3['RA'][list9_c_sp],'DEC':df_2_list3['DEC'][list9_c_sp],
                        'Telescope_used':df_2_list3['Telescope_used'][list9_c_sp],'Notes':df_2_list3['Notes'][list9_c_sp],
                         'observation_dates':df_2_list3['observation_dates'][list9_c_sp]
                        })

list9_d_sp = random.sample(list(df_2_list4.index.values),5) 

d = pd.DataFrame({'Sp_ID':df_2_list4['Sp_ID'][list9_d_sp],'Gaia_ID':target_list_v6['Gaia_ID'][list9_d_sp],'nb_hours_surved':df_2_list4['nb_hours_surved'][list9_d_sp],
                       'Program':df_2_list4['Program'][list9_d_sp],'RA':df_2_list4['RA'][list9_d_sp],'DEC':df_2_list4['DEC'][list9_d_sp],
                        'Telescope_used':df_2_list4['Telescope_used'][list9_c_sp],'Notes':df_2_list4['Notes'][list9_d_sp],
                         'observation_dates':df_2_list4['observation_dates'][list9_d_sp]
                        })

pd.concat([a,b,c,d])

In [ ]:
df_2_list9 = pd.DataFrame({'Sp_ID':all_targets['Sp_ID'][list9],'Gaia_ID':target_list_v6['Gaia_ID'][list9],'nb_hours_surved':all_targets['nb_hours_surved'][list9],
                       'Program':all_targets['Program'][list9],'RA':all_targets['RA'][list9],'DEC':all_targets['DEC'][list9],
                        'Telescope_used':all_targets['Telescope_used'][list8],'Notes':all_targets['Notes'][list9],
                         'observation_dates':all_targets['observation_dates'][list9]
                        })
df_2_list9.to_csv('List8.csv',sep=',',index=None)


df_2_list9

In [ ]:
np.array(list(list7))

In [ ]:
target_list_v6

## Others

In [ ]:
lc = getSPCdata(target='Sp1501+2250', date='*', ap=6, user="educrot", password="58JMSGgdmzTB")


## Script  for Artem (stats ARtemis)

In [ ]:
idx_artemis = np.where((np.array([target_list_v6['telescope'][i].find('Artemis') for i in range(len(target_list_v6))]) != -1))[0]



In [ ]:
target_list_v6['Sp_ID'][idx_artemis]